# Data Explanations and Post-Processing Scripts for MimiGIVE.jl

The following notebook will detail the different data files in the _data_ folder including their source and any post-processing done to obtain the files directly imported by the `MimiGIVE` API.

## 1. General Model Data

**Dimension_countries.csv**

This file holds the list of countries, by ISO3 code, that serve as the `country` dimensions for this model.  By necessity of obtaining country-level GDP and Population, this list must be a subset of the countries available to be provided by the `Socioeconomics` component utilized, which may be either from [MimiSSPs](https://github.com/anthofflab/MimiSSPs.jl) or [MimiRFFSPs](https://github.com/rffscghg/MimiRFFSPs.jl). 184 countries are available in Benveniste et al., 2020, and are subset for components which do not have calculations available for all countries (CIAM calculates for 145 of the total countries).

Further information on country names, numeric codes, and alphanumeric codes for the ISO3 dataset can be found [here](https://en.wikipedia.org/wiki/ISO_3166-1) on Wikipedia. 

## 2. Mortality Components

### 2a. General

**Mortality_cdf_spp_country_extensions.csv** and **Mortality_cdf_ssp_country_extensions_annual.csv**

_Mortality_cdf_spp_country_extensions.csv_ was recieved from David Smith (EPA) on October 8, 2021, and holds the SSP mortality data with extensions out to 2300. The raw data out to 2100 are stored at the [Wittgenstein Centre](https://nam02.safelinks.protection.outlook.com/?url=http%3A%2F%2Fdataexplorer.wittgensteincentre.org%2Fwcde-v2%2F&data=04%7C01%7Crennert%40rff.org%7C96cba6d9096647c0159f08d98a001bc0%7Cb29f848db9144be4ad1f89bdbdb8030a%7C0%7C1%7C637692555407649003%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C2000&sdata=Ji7h6eHB2mHnEaCco3RvbvpDP1j4PMcNHdRa2pdgMYQ%3D&reserved=0) and can be accessed in R by using the [wcde](https://guyabel.github.io/wcde/) package. Samir KC and Wolfgang Lutz, the two original authors of the population projections, are jointly appointed to IIASA and to the Wittgenstein Center. This database only has data for SSPs 1 through 3. Based on [KC and Lutz et al., 2017](https://www.sciencedirect.com/science/article/pii/S0959378014001095)it appears that the mortality assumptions are similar for SSP2 and SPP4 and for SSP1 and SSP5, this assumption is adopted and SSP1 is used as a proxy for SSP5 and SSP2 as a proxy for SSP4, for this variable. For the projections, the crude death rate post-2100 for each country converging towards the 2100 world crude death (by SSP). All countries reach the 2100 world crude death rate in 2300. Data exists for 200 countries. Units for the crude death rate are number of deaths occurring over a five-year period (period ends on the year in the year column) per 1,000 total population.

Post processing of these data for our use is as follows, and produces _Mortality_cdf_ssp_country_extensions_annual.csv_. To downscale these data from five year increments to annual deaths per 1000 people, the interval value is divided value by the population in the middle of the interval, multiply by 1000 (people), and divide by the time interval (5 years). Note that because of this recommended post processing step this needs to be done ahead of time for each potential population scenario that will be employed, and adjusted if population scenarios change.

In [ ]:
using DataFrames, Query, CSVFiles

mortality_countries = load(joinpath(@__DIR__, "..", "data", "Dimension_countries.csv")) |> DataFrame
time_interval = 5

raw_df = load(joinpath(@__DIR__, "..", "data", "Mortality_cdr_spp_country_extensions.csv")) |> 
    DataFrame |>
    @filter(_.alpha3 in mortality_countries.CountryISO) |>
    DataFrame

# check that we have all the countries
sort(unique(raw_df.alpha3)) == sort(mortality_countries.CountryISO) ? nothing : error("Baseline mortality data needs to include all mortality countries used in model.")

# interpolate each country for each SSP
annual_df = DataFrame(:year => [], :ISO => [], :scenario => [], :cdf => [])
for SSP in unique(raw_df.scenario_abb)
    for country in unique(raw_df.alpha3)

        filtered_df = raw_df |> @filter(_.scenario_abb == SSP && _.alpha3 == country) |> DataFrame
        population = load(joinpath(@__DIR__, "..", "data", "Benveniste_SSPs", "Benveniste_$SSP.csv")) |> 
            DataFrame |>
            @filter(_.country == country) |> 
            DataFrame
        
        for year in unique(filtered_df.year)
            
            # get the end of interval cdr value
            cdr = ((filtered_df |> @filter(_.year == year) |> DataFrame).cdr)[1]
            # get population in the years of this time period for scaling
            pop_interval = (population |> @filter(_.year in collect(year-4:year)) |> DataFrame).pop
            # get the deaths per five year period, divide by population in center year, and scale to 1000
            deaths_in_interval = sum(pop_interval/1000) * cdr
            # get annual cdr for the interval
            cdf_annual = ((deaths_in_interval/pop_interval[3]) / 5) * 1000
            
            append!(annual_df, DataFrame(
                :year => collect(year - 4:year),
                :ISO => fill(country, 5),
                :scenario => fill(SSP, 5),
                :cdf => fill(cdf_annual, 5)
            ))
        end
    end
end

annual_df |> save(joinpath(@__DIR__, "..", "data", "Mortality_cdr_spp_country_extensions_annual.csv"))

### 2b. Cromar Mortality Component

**Dimension_cromar_mortality_regions.csv**

This file holds the list of regions used for the Cromar Mortality damage functions, as found in Cromar et al. and that serve as the `cromar_regions` dimension for this model.

**CromarMortality_damages_coefficients.csv**

This file holds the coefficients received from Cromar et al. for the damage function.

**Mapping_countries_to_cromar_mortality_regions.csv**

This file holds the mapping from the `country` dimensoin in the model to the `cromar_mortality_regions` dimension in the model. This was done by mapping the GCAM regions used by the `gcam_regions` to the `cromar_regions` as follows and then using the country lists from GCAM.

## 3. Agriculture Component

**Dimension_fund_regions.csv**

This file holds the list of codes for the [FUND](https://github.com/fund-model/MimiFUND.jl) regions that serve as the `fund_regions` dimensions for this model.

**Mapping_countries_to_fund_regions.xlsx**

This workbook holds the working lookups and crosswalks to move between various subsets of countries and FUND regions, including country names in addition to ISO3 codes etc.  It is the source for the following file.

**Mapping_countries_to_fund_regions.csv**

This file holds the mapping from the `country` dimension in the model to the `fund_regions` dimension in the model.  It is pulled directly from the _BENVENISTE_ tab in the workbook above, _Mapping_countries_to_fund_regions.xlsx_.

**Benveniste_SSPs/Benveniste_SSPX.csv** and **Benveniste_SSPs/Agriculture_1990vals.csv**

The _data/Benveniste_SSPs_ folder holds copies of the the Benveniste et al., 2020 data used in the `MimiSSPs` component for the setting of `SSPmodel = "Benveniste"` filtered for the year 1990.  This is processed into regional values for the year 1990 to provide for the `pop90` and `gdp90` parameters for the `Agriculture` component using the script below.  Notice this uses _ALL_ 184 countries, and could be generalized to accomodate a different countries list if needed in the future.

In [ ]:
using DataFrames, Query, CSVFiles

input_output_mapping = load(joinpath(@__DIR__, "..", "data", "Mapping_countries_to_fund_regions.csv")) |> DataFrame |> @select(:ISO3, :fundregion) |> DataFrame
fund_regions = load(joinpath(@__DIR__, "..", "data", "Dimension_fund_regions.csv")) |> DataFrame 

df_1990 = DataFrame(:SSP => [], :fund_region => [], :pop => [], :gdp => [])

for SSP in ["SSP1", "SSP2", "SSP3", "SSP4", "SSP5"]
    df_SSP = load(joinpath(@__DIR__, "..", "data", "Benveniste_SSPs", "Benveniste_$SSP.csv")) |> 
        DataFrame |>
        @filter(_.year == 1990) |>
        DataFrame

    for region in fund_regions.fund_region
        countries = (input_output_mapping |> @filter(_.fundregion == region) |> DataFrame |> @select(:ISO3) |> DataFrame |> Matrix)[:]
        df_SSP_region = df_SSP |> @filter(_.country in countries) |> DataFrame

        if region == "SEA" # note that TWN has a missing GDP value until 2015 use value from imf.org ("Report for Selected Countries and Subjects". www.imf.org. Archived from the original on 31 July 2020. Retrieved 2 May 2020.)
            TWN_row = findfirst(i -> i == "TWN", df_SSP_region.country)
            df_SSP_region.gdp[TWN_row] = 205 * 1.4 # $2020 to $2005
        end

        append!(df_1990, DataFrame(:SSP => SSP, :fund_region => region, :pop => sum(df_SSP_region.pop), :gdp => sum(df_SSP_region.gdp)))
    end
end
df_1990 |> save(joinpath(@__DIR__, "..", "data", "Benveniste_SSPs", "Agriculture_1990vals.csv"))

## 4. Sea Level Rise Component (CIAM)
**Dimension_ciam_countries.csv**

This file holds the list of countries, by ISO3 code, that serve as the `ciam_country` dimensions for this model and are a subset of what is used in [MimiCIAM.jl](https://github.com/raddleverse/MimiCIAM.jl), representing only the 145 countries which overlap with our `country` dimension. 

**CIAM/diva_segment_latlon.csv**

This file holds metadata for the segments used in CIAM and was puleld directly frim the [CIAM repository](https://github.com/raddleverse/MimiCIAM.jl/tree/master/data), and subsequently the `rgn` ISO3 code was joined based on the MimiCIAM key file `xsc.csv`, for all available segments.

**CIAM/FINGERPRINTS_SLANGEN_Bakker.nc**

This file holds the BRICK original fingerprints from the [BRICK repository](https://github.com/scrim-network/BRICK/raw/master/fingerprints) for all available segments.

**CIAM/segment_fingerprints.csv**

This file is created using scripts in `src/utils/lsl_downscaling.jl` as called below.

In [ ]:
using MimiGIVE

MimiGIVE.get_segment_fingerprints()


**CIAM/xsc_ciam_countries.csv**

This file contains metadata about the segments, including seg (segment name), rgn (region of segment ID), greenland (in or out), island (yes or no), segID (ID of segment),rgnID (ID of region) **that are used in this model**.  This will perfectly match with the `ciam_country` dimension, and does not necessarily thus include all countries available within `MimiCIAM.jl`.

## 5. Climate Components - FAIR v1.6.2

**FAIR_ar6/AR6_emissions_sspXX_1750_2300.csv**

These data are obtained directly from [MimiFAIRv1_6_2.jl](https://github.com/FrankErrickson/MimiFAIRv1_6_2.jl) and represent the emissions trajectories used by AR6.

**FAIR_mcs/fair-1.6.2-wg3-params.json.zip**

This file is obtained directly from the [MimiFAIRv1_6_2.jl](https://github.com/FrankErrickson/MimiFAIRv1_6_2.jl) which in turn downloaded [from Zenodo](https://zenodo.org/record/5513022#.YWPzVS2cYW9) and represents the constrained parameter set used in AR6.  Post-processing of these data is as follows, to be used in the Monte Carlo Simulation. Processing can be found in the script _process_fair_mcs_params.jl_, which will first require unzipping _fair-1.6.2-wg3-params.json.zip_.

In [ ]:
include(joinpath(@__DIR__, "..", "utils", "process_fair_mcs_params.jl"))

## 6. Energy Component

**energy_damages_gcam_region_coefficients.csv**

This file holds the coefficients used in the energy damage function, with one coefficient per GCAM `gcam_energy_regions` as defined in the following file.

**Dimension_gcam_energy_regions.csv**

This file holds the names of the GCAM energy regions.

**Mapping_countries_to_gcam_energy_regions.csv**

This file maps the 184 model `countries` to the 12 GCAM `gcam_energy_regions`, noting that the following mappings are made manually as they were not explicitly included in the input data:

```
HKG	 Hong Kong    China
MAC	 Macao	      China
MMR	 Burma	      Other_Asia
PRI	 Puerto Rico  USA
```

## 7. VSL Component

There are two sources for the value of statistical life (VSL) being used in the model. These can be selected by the user when calling `get_model()` with the keyword option: `vsl=:fund` or `vsl=:epa`.  _There are no data files associated with these choices, only hard coded values, but we include detail here to be comprehensive._

- The symbol `:fund` uses a VSL that is drawn from the original [FUND documentation](http://www.fund-model.org/files/documentation/Fund-3-9-Scientific-Documentation.pdf) citing Cline (1992): _"This calibration results in a best guess value of a statistical life that is 200 times per capita income (Cline, 1992)."_ Under this framework, the VSL in 2005 U.S. dollars for income per capita <img src="https://render.githubusercontent.com/render/math?math=y"> in model year <img src="https://render.githubusercontent.com/render/math?math=t"> for country <img src="https://render.githubusercontent.com/render/math?math=c"> is: 

    $$vsl_{tc} = \$6,082,532.05 \times (\dfrac{y_{tc}}{\$30,412.66}).$$ 
    
    Using this formula results in a baseline VSL for the U.S. in 2017 of $9,847,389.53 (2011 U.S. dollars).

<br>

- The symbol `:epa` uses a 2020 U.S. VSL as the base for the willingness to pay for reductions in mortality risk. This starting VSL is developed according to the most recent U.S. EPA Guidelines for Economic Analyses ([U.S. EPA, 2010](https://www.epa.gov/sites/default/files/2017-09/documents/ee-0568-22.pdf)). This builds on the 1990 U.S. EPA VSL of $4.8 million (1990 U.S. dollars) and adjusts for inflation and income growth using the following equation:

     $$vsl_{2020,U.S.} = (\alpha_{1990} \times \gamma) \times (\dfrac{y_{2020,U.S.}}{y_{1990,U.S.}})^\varepsilon.$$
    
    Here, <img src="https://render.githubusercontent.com/render/math?math=\alpha_{1990}">=$4,800,000 (1990 U.S. dollars).
    
    U.S. EPA Guidelines use the Bureau of Economic Analysis' (BEA) Gross Domestic Product implicit price deflator to adjust across dollar-years ([U.S. BEA Table 1.1.9, 2021](https://www.bea.gov)). Using this inflation index, <img src="https://render.githubusercontent.com/render/math?math=\gamma=\frac{87.504}{63.631}">=1.375178765.    
    
    Income per capita in the U.S. comes from the U.S. BEA's real gross domestic product per capita series ([U.S. BEA, 2021](https://fred.stlouisfed.org/series/A939RX0Q048SBEA#0)). <img src="https://render.githubusercontent.com/render/math?math=y_{2020}">=$55,685.00 (2012 U.S. dollars). 
    
    Using the same BEA series for the starting year, <img src="https://render.githubusercontent.com/render/math?math=y_{1990}">=$37,460.00 (2012 U.S. dollars). 
        
    U.S. EPA Guidelines provide a range of estimates for the income elasticity of VSL, 0.08, 0.4, and 1 ([U.S. EPA, 2010](https://www.epa.gov/sites/default/files/2017-09/documents/ee-0568-22.pdf), [BenMAP 2021](https://www.epa.gov/sites/default/files/2015-04/documents/benmap-ce_user_manual_march_2015.pdf)). As stated in the Guidelines: 
    
     _"The economics literature shows that the income elasticity of WTP to reduce mortality risk is positive, based on cross-sectional data. As a result, benefits estimates of reduced mortality risk accruing in future years may be adjusted to reflect anticipated income growth, using the range of income elasticities (0.08, 0.40 and 1.0) employed in The Benefits and Costs of the Clean Air Act, 1990-2010. Footnote: For details see [Kleckner and Neuman (2000)](https://www3.epa.gov/ttn/ecas/regdata/Benefits/background/klecknerandneumann1999.pdf)."_ Additional relevant citations: [The Benefits and Costs of the Clean Air Act: 1990-2010. EPA 410-R-99-001](https://www.epa.gov/clean-air-act-overview/benefits-and-costs-clean-air-act-1990-2010-first-prospective-study).
        
    In practice and used here, the U.S. EPA uses the central value in regulatory impact analyses in the U.S. such that <img src="https://render.githubusercontent.com/render/math?math=\varepsilon">=0.4:
    
    Using this formula results in a baseline VSL for the U.S in the year 2020 of $7,735,147.07 (2005 U.S. dollars). 
    
    One additional transformation is made to bring income per capita in 2020 ([U.S. BEA, 2021](https://fred.stlouisfed.org/series/A939RX0Q048SBEA#0)) in 2012 U.S. dollars to 2005 U.S. dollars (the model's dollar year). Using this inflation index, noting that the base year of the index is 2012, <img src="https://render.githubusercontent.com/render/math?math=\gamma=\frac{87.504}{100}">=0.87504 resulting in 2020 income per capita in 2005 U.S. dollars of $48,726.60. From here, the following equation is used to adjust the VSL intertemporally for year <img src="https://render.githubusercontent.com/render/math?math=t"> and cross-sectionally for country <img src="https://render.githubusercontent.com/render/math?math=c"> using income per capita <img src="https://render.githubusercontent.com/render/math?math=y">:

     $$vsl_{tc} = \$7,735,147.07 \times (\dfrac{y_{tc}}{\$48,726.60})^\varepsilon.$$

     Here, the income elasticity of VSL <img src="https://render.githubusercontent.com/render/math?math=\varepsilon">=1 is used to adjust for different income levels across countries and changes in income through time.  

## 8. BRICK Sea Level Rise

**BRICK_posterior_parameters_10k.csv**

This file holds 10,000 sets of posterior parameters for BRICK uncertainty post-calibration.

## 9. Constraints

### Imposing Constraints on the RFF-SPs for use in MCS

In the forthcoming [BPEA paper](https://www.rff.org/publications/working-papers/the-social-cost-of-carbon-advances-in-long-term-probabilistic-projections-of-population-gdp-emissions-and-discount-rates), one constraint that is imposed as a sensitivity<sup>1</sup> is to drop trials where global average income per capita in 2300 falls in the 1% tails of distribution. This results in 9,800 remaining trials (dropping 200). The _rffsps_1ct_trimmed.csv_ file holds this constrained sample of 9,800 trial IDs.

<br><br>

---
1. The motivation behind this sensitivity is described in [the paper](https://www.rff.org/publications/working-papers/the-social-cost-of-carbon-advances-in-long-term-probabilistic-projections-of-population-gdp-emissions-and-discount-rates) on page 36 as: "The second row of Table 1 highlights this greater stability under stochastic discounting by showing a sensitivity case in which we drop the top and bottom 1 percent of the global average income trajectories" along with footnote number 34: "Specifically, we drop the draws with global average GDP per capita in 2300 in the top 1percent and bottom 1 percent of draws, before taking the average SCC."

In [ ]:
## cleans RFF-SPs, aggregates to global, and trims outliers from upper and lower 1% 
## of global average income distribtion in the year 2300
 
##########################
#################  library
##########################
  
## Clear worksace
rm(list = ls())
gc()
  
## This function will check if a package is installed, and if not, install it
list.of.packages <- c('magrittr','tidyverse','arrow','zoo')
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "http://cran.rstudio.com/")
lapply(list.of.packages, library, character.only = TRUE)
  
##########################
##################### data
##########################
  
## inflate rff data
inflator = 113.648/87.504 # 2005$ to 2020$, recovered 9/16/2021 from: https://apps.bea.gov/iTable/iTable.cfm?reqid=19&step=3&isuri=1&select_all_years=0&nipa_table_list=13&series=a&first_year=2015&last_year=2020&scale=-99&categories=survey&thetable=
rffsp_inflator = (87.504/98.164)*inflator #rff-sps are in 2011$. Convert to 2005$ then to $2020$, from https://github.com/rffscghg/MimiRFFSPs.jl/blob/9f78ab45e74040134152612eb6c31327c6d1543a/src/components/SPs.jl#L5
  
## list of files
files <- list.files('.julia/datadeps/rffsps_v3/pop_income', full.names=T)
  
## blank data
data = tibble()

## start loop
for (i in 1:length(files)){

    print(paste('Reading file number ', i, ' of ', length(files)))
  
    file = files[i]
    rffsp.id = str_sub(str_split(file,pat='/')[[1]][8],22,-9)

    rff = read_feather(file) %>% 
      rename_all(tolower) %>% 
      group_by(country) %>%
      complete(year=seq(first(year), last(year))) %>% 
      mutate(pop = exp(na.approx(log(pop))),
             gdp = exp(na.approx(log(gdp)))) %>% 
      ungroup() %>% 
      select(-country) %>% 
      group_by(year) %>% 
      summarise_all(sum,na.rm=T) %>% 
      mutate(pop=pop/1e6,
             gdp=gdp/1e6*rffsp_inflator,
             ypc=gdp/pop,
             ypc.gr=log(ypc/lag(ypc,default=ypc[1])),
             rffsp.id=rffsp.id) %>% 
      relocate(rffsp.id,year,gdp,pop,ypc,ypc.gr)

    data = bind_rows(data,rff)
    
    rm(rff)
    gc()

} 
  
## export global untrimmed aggregates
data %>% write_csv(.,'RFFModel/data/rffsp_global.csv')

## 
rff = read_csv('data/rff/rffsp_global.csv') %>% filter(year==2300) %>% rename(trialnum=rffsp.id) %>% select(trialnum,ypc) %>% arrange(trialnum)

## trim outliers based on ypc in 2300
rff_trimmed = rff %>% filter(ypc>=quantile(ypc,0.01) & ypc<=quantile(ypc,0.99))

## check outliers
rff_outliers = rff %>% filter(ypc<quantile(ypc,0.01) | ypc>quantile(ypc,0.99))

## export outliers for use in mcs
rff_trimmed %>% select(trialnum) %>% arrange(trialnum) %>% write_csv('RFFModel/data/rffsp_1pct_trimmed.csv')